In [1]:
from pyzotero import zotero
from zotero2qmd.zotero2qmd import item2main, main2qmd, filter_pubs
from collections import Counter
import re
import yaml
from pathlib import Path

In [2]:
keypath = Path(".zotero")
with keypath.open() as keyfile:
    key_text = keyfile.readline().strip()

In [3]:
zot = zotero.Zotero(library_id='7642731', library_type='user', api_key=key_text)

In [4]:
pubs = zot.publications()

In [5]:
filtered_pubs = filter_pubs(pubs)

In [6]:
all_mains = [item2main(x) for x in filtered_pubs]

In [7]:
def make_file_names(all_mains):
    first_aut = [x["author"][0]["name"]["family"] for x in all_mains]
    years = [x["date"].split("-")[0] for x in all_mains]
    all_init = ["".join(re.findall(pattern=r'[A-Z]',string=x["description"])) for x in all_mains]
    all_stem = [f"{aut}_{year}_{init}" for aut, year, init in zip(first_aut, years, all_init)]
    all_stem = [re.sub(r"_$", "", x) for x in all_stem]
    return all_stem

In [8]:
all_mains

[{'params': {'notes': ''},
  'author': [{'name': {'given': 'James', 'family': 'Tanner'}},
   {'name': {'given': 'Morgan', 'family': 'Sonderegger'}},
   {'name': {'given': 'Jane', 'family': 'Stuart-Smith'}},
   {'name': {'given': 'Josef', 'family': 'Fruehwald'}}],
  'title': 'Toward “English” Phonetics: Variability in the Pre-consonantal Voicing Effect Across English Dialects and Speakers',
  'date': '2020-5-29',
  'date-format': 'YYYY',
  'doi': '10.3389/frai.2020.00038',
  'description': 'Frontiers in Artificial Intelligence',
  'abstract': 'Recent advances in access to spoken-language corpora and development of speech processing tools have made possible the performance of “large-scale” phonetic and sociolinguistic research. This study illustrates the usefulness of such a large-scale approach—using data from multiple corpora across a range of English dialects, collected, and analyzed with the SPADE project—to examine how the pre-consonantal Voicing Effect (longer vowels before voiced 

In [9]:
all_stems = make_file_names(all_mains)

In [10]:
out_path = Path("research","papers")

In [11]:
for stem, item in zip(all_stems, all_mains):
    out_file = out_path.joinpath(stem).with_suffix(".qmd")
    if not out_file.exists():
        with out_file.open(mode = "w"):
            qmd_string = "---\n"+yaml.dump(item)+"\n---"
            out_file.write_text(qmd_string)